<a href="https://colab.research.google.com/github/wadeyman/BT4212/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BT4212 Homework 4
## Search Engine Optimization and Analytics

Term: Fall 2023

*Individual Assignment, due Nov 5，5PM*

## Submission Instruction

This homework contains several coding tasks and short-answer questions to explore several predictive models for page-rank. For coding part, please write codes in the corresponding cells. I may provide some comment lines as guideline. For short-answers, type your answer in the cells with **ANSWER: HERE**. Please double click those cells and directly input your answer.

I recommend you use Python 3 for this homework. Python 2 may not be supported. You can use either your own PC or Google Colab to do this homework. GPU support is NOT required.


Save your notebook `.ipynb` file as `StudentID_YourName_HW4.ipynb`. Generate an `.html` file from `.ipynb` file and save as `StudentID_YourName_HW4.html`. Zip your notebook file and html file into a single `.zip` file.

Upload the `zip` file as `StudentID_YourName_HW4.zip`. Please DO NOT include data file in your zip file (too large to upload and download).

**Please make sure your code is executable.**

If you are using Google Colab,
* Please make sure you have expanded all hidden cells. You can refer to https://stackoverflow.com/questions/62457417/unhide-all-cells-in-google-colab for more information.
* How to generate an HTML file from your notebook file in Google Colab? Please refer to https://stackoverflow.com/questions/53460051/convert-ipynb-notebook-to-html-in-google-colab

HW4 is worth 80 points in total.

In [41]:
# Input your name and studentID
name = "Tan Hui Rong"
stuID = "A0216246J"

In [42]:
# Import packages
# the recommended version is listed but you could try using the most updated one.
import numpy as np
import pandas as pd

import statsmodels.api as sm # recommended version: 0.13.0
from statsmodels.miscmodels.ordinal_model import OrderedModel

import xgboost as xgb # recommended version: 1.5.0
from xgboost import plot_importance

from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor



In [43]:
# To fix random seeds (Note that you may still get slightly different results.)
np.random.seed(12345)

# To ignore some warnings
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)


In [44]:
## If you are using Google Colab and get an error regarding statsmodels(OrderedModel),
## you can use the following line to re-install statsmodels package.
## This will takes some time, please RESTART the RUNTIME after installation.

# !pip install statsmodels==0.13.0

## You may also want to mount your Google Drive to allow easy file loading.

# from google.colab import drive
# drive.mount('/content/drive')

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Background Information: About the Data
The data set is acquired in the following steps.
1. Identify a set of 20 keywords.
2. For each keyword, search in google, and return the first 98 websites. It has $20\times98=1960$ observations in total.
3.  Split the dataset into training and test (or validation) data. The training data includes $70\%$ of observations ($14\times 98=1372$ rows) while the test one has $30\%$ of observations ($6\times 98=588$ rows).

There are two data files. `Train_dta.csv` for training data and `Test_dta.csv` for testing data. Open the data file with Excel may encounter some unexpected errors. Simply download another copy from Canvas, if it occurs.

### Load Data

In [46]:
# Load the training and test datasets and print the first 5 rows of the training dataset.
# Please use pd.read_csv("data.csv") to load your data
# Run this cell

feature_col =["TitleFlag", "TitleDensity", "URLFlag", "URLDensity", "MetaFlag", "MetaDensity", "PageAuthority", "DomainAuthority", "LinkingDomain", "InboundLink" , "RankingKeyword" ]
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Train_dta.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Test_dta.csv")


In [47]:
train_data.head()

,ID,Position,ReverseRank,Title,URL,Meta,TitleFlag,TitleDensity,URLFlag,URLDensity,MetaFlag,MetaDensity,PageAuthority,DomainAuthority,LinkingDomain,InboundLink,RankingKeyword
0,1,1,100,Job Interview Questions Singapore: Answers to ...,https://blog.carousell.com/job-interview-quest...,sep 19 2017 common job interview questions in ...,0,0.3000,0,0.4286,0,0.1017,32,60,4,5.0,4
1,2,2,99,Job Interview Questions - Hudson Singapore,https://www.hudson.sg/career-advice/job-interv...,discover the five most common job interview qu...,0,0.6000,0,0.2500,0,0.2174,19,30,0,0.0,9
2,3,3,98,20 tips that will help you ace your next job i...,http://www.asiaone.com/business/20-tips-will-h...,oct 8 2017 with singapore's unemployment rate ...,0,0.2143,0,0.2143,0,0.0645,40,80,3,400.0,0
3,4,4,97,Common interview questions and answers | Rober...,https://www.roberthalf.com.sg/career-advice/in...,knowing what job interview questions you might...,0,0.1429,0,0.1111,0,0.2500,22,40,0,0.0,4
4,5,5,96,10 Common Interview Questions and How to ... -...,http://gradsingapore.com/graduate-careers-advi...,don't be caught off guard in an interview try ...,0,0.2000,0,0.0714,0,0.1429,24,33,2,2.0,6


There are many columns in the data, e.g., title, url and meta desciptions. The detailed information about each column is in the appendix.

**We will only use: “TitleFlag”, “TitleDensity”, “URLFlag”, “URLDensity”, “MetaFlag”, “MetaDensity”, “PageAuthority”, “DomainAuthority”, “LinkingDomain”, “InboundLink” and “RankingKeyword” as features, "ReverseRank" as label.**

In [48]:
# Split feature and label
# Run this cell

train_feature = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Train_dta.csv",usecols=feature_col)
train_label = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Train_dta.csv",usecols=["ReverseRank"])

test_feature = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Test_dta.csv",usecols=feature_col)
test_label = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Test_dta.csv",usecols=["ReverseRank"])


In [49]:
# Run this cell
print(train_feature.shape,
      train_label.shape,
      test_feature.shape,
      test_label.shape)

(1372, 11) (1372, 1) (588, 11) (588, 1)


In [50]:
# Run this cell
# This serves as your data input
X_train = train_feature # trainig feature, as a dataframe in pandas
X_test = test_feature # test feature, as a df

y_train = train_label.values # training label, as a numpy array
y_test = test_label.values # test label, as a np array


To avoid potential issues of shallow copy and deep copy. Try to load data separately for each problem, although they may be the same.

## Problem 1 Pointwise Rank. (35 points)

###  Q1. Linear Regression (10 points)
Please use the training dataset to fit a linear regression with all variables aforementioned. Please use the trained linear regression model to predict the rank in the test dataset and **report the root mean square error (RMSE) of the prediction**, i.e., $\sqrt{\frac{1}{n}\sum_{i=1}^n(y_i-\hat{y}_i)^2}$, where $n$ is the total number of data points in the test dataset, $y_i$ is the true value of the outcome variable in the test dataset, and $\hat{y}_i$ is the model prediction of the outcome variable. (5 points)

Reference: https://www.statsmodels.org/dev/examples/notebooks/generated/ols.html

In [51]:
# Data Input
X_train = train_feature # trainig feature, as a dataframe in pandas
X_test = test_feature # test feature, as a df

y_train = train_label.values # training label, as a numpy array
y_test = test_label.values # test label, as a np array

In [52]:
# Model: USE sm.OLS(y_train,X_train)
# Fit: USE model.fit()
# y_train is a numpy array from train_label, X_train is a pandas dataframe from train_feature.
# Remember to add an intercept by sm.add_constant() to both train and test data!
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)
lin_model = sm.OLS(y_train, X_train)
results = lin_model.fit()

In [53]:
# Print model summary by model.summary()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     14.39
Date:                Thu, 26 Oct 2023   Prob (F-statistic):           1.33e-26
Time:                        17:33:42   Log-Likelihood:                -6457.1
No. Observations:                1372   AIC:                         1.294e+04
Df Residuals:                    1360   BIC:                         1.300e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              32.4841      3.072     10.575      0.000      26.458      38.510
TitleFlag          11.0206      4.107      2.683      0.007       2.964      19.078
TitleDensity       15.7273      5.382      2.922      0.004       5.168      26.286
URLFlag            -2.5156      4.326     -0.581      0.561     -11.003       5.972
URLDensity         34.8241      7.117      4.893      0.000      20.862      48.786
MetaFlag           -3.3674      2.005     -1.680      0.093      -7.300       0.565
MetaDensity        18.6692     11.354      1.644      0.100      -3.605      40.943
PageAuthority       0.0907      0.123      0.737      0.461      -0.151       0.332
DomainAuthority     0.1167      0.055      2.122      0.034       0.009       0.225
LinkingDomain      -0.0012      0.001     -1.342      0.180      -0.003       0.001
InboundLink     -1.855e-05   2.88e-05     -0.645      0.519    -7.5e-05    3.79e-05
RankingKeyword     -0.0043      0.003     -1.342      0.180      -0.011       0.002
==============================================================================
Omnibus:                      294.603   Durbin-Watson:                   0.264
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               58.136
Skew:                          -0.077   Prob(JB):                     2.38e-13
Kurtosis:                       2.004   Cond. No.                     7.44e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.44e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [54]:
# Make prediction
predictions = results.predict(X_test)
predictions[:]

0      53.664124
1      56.095566
2      66.037076
3      65.902540
4      52.730487
         ...    
583    42.524573
584   -18.311877
585    37.341913
586    65.034483
587    37.293170
Length: 588, dtype: float64

In [55]:
y_test = pd.Series(pd.DataFrame(y_test)[0])

In [56]:
type(predictions)

pandas.core.series.Series

In [57]:
# Print RMSE, y_test is a numpy array from test_label, y_pred is a numpy array from your model prediction
resids = y_test - predictions
mse = np.mean(resids**2)

rmse = np.sqrt(mse)

print("RMSE:", rmse)

RMSE: 37.150724570848375


**Pick the most statistically significant variable and interpret its estimated beta coefficient.** (5 points)

*ANSWER: HERE*

URL density gives a P-value closest to 0, suggesting it is the msot statistically significant variable. The estimated beta coefficient is 34.8241. This means for every 1 unit increase in URL density, we see a positive change in dependent variable ReverseRank, holding all other variables constant.


### Q2. Logistic Regression (15 points)

For the baseline logistic regression, it does not require an ordinal relationship among the levels of the outcome variable, e.g., level 1 does not necessarily imply superiority or inferiority compared with level 2. However, for ordinal variable, its levels can be ranked implying a higher value than other level, e.g., in the school grade, A is better than B. We will use **ordinal** logistic regression for this problem.

Please use the training dataset to fit an **ordinal** logistic regression model with all variables aforementioned. **Use the `ReverseRank` as the outcome variable**, that is apparently a ranked variable**. (5 points)

Note that you may not be able to run ordinal logistic regression because the distributions of several variables are too skewed. You can use `new_variable = np.log(the_problematic_variable+1)` to **transform those variables in both training and testing data.** (5 points)

Please use the trained ordinal logistic regression model to predict the rank in the test dataset. **Print the predicted rank on the test dataset and report the RMSE of the prediction.** （5 points）

Reference: https://www.statsmodels.org/dev/examples/notebooks/generated/ordinal_regression.html.
https://stats.oarc.ucla.edu/r/dae/ordinal-logistic-regression/.

Remarks: You will find tons of useful materials about statistical modelling in UCLA website, even though mostly implemented with R or Stata.

In [58]:
# Data Input
X_train = train_feature # trainig feature, as a dataframe in pandas
X_test = test_feature # test feature, as a df

y_train = train_label.values # training label, as a numpy array
y_test = test_label.values # test label, as a np array

Variables with high skewness values (highly positive or negative) may benefit from log transformation to reduce the skewness. Highly skewed variables can affect the performance and interpretability of some statistical models. In this case, linear models assume homoskedastic being that the residuals (the differences between the observed and predicted values) are normally distributed and have constant variance. Skewed data violates this assumption, leading to unreliability.

TitleFlag, URLFlag, LinkingDomain, InboundLink, RankingKeyword variables have skewness values above 1 indicating significant positive skewness. Shall leave MetaFlag and MetaDensity as they are moderately skewed and we want to retain as much of the original data.

In [59]:
# Adjust your input
# Please specify which parts of data you do a log transformation on.
# There are many ways to determine the skewness,
# e.g., plotting the distribution, calling pandas.DataFrame.skew, etc.
# As long as you provide reasons for the transformation, you will be awarded the points.
pd.DataFrame.skew(X_train)

TitleFlag           4.018238
TitleDensity        0.755273
URLFlag             4.343633
URLDensity          0.872202
MetaFlag            1.733323
MetaDensity         1.122730
PageAuthority       0.362187
DomainAuthority     0.187663
LinkingDomain      15.575515
InboundLink        16.606262
RankingKeyword     12.241945
dtype: float64

In [60]:
columns_to_log = ["TitleFlag", "URLFlag", "LinkingDomain", "InboundLink", "RankingKeyword"]
X_train_withLogs = X_train
X_train_withLogs[columns_to_log] = X_train_withLogs[columns_to_log].apply(lambda x: np.log(x + 1))

In [61]:
# Model: use OrderedModel(y_train, X_train, distr='logit')
# Fit: use model.fit(method='bfgs', disp=False)
# y_train is a numpy array from train_label, X_train is a pandas dataframe from train_feature.
ord_model = OrderedModel(y_train, X_train_withLogs)
results = ord_model.fit(method='bfgs', disp=False)

In [62]:
# Print model summary by model.summary()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                      y   Log-Likelihood:                -6220.9
Model:                   OrderedModel   AIC:                         1.266e+04
Method:            Maximum Likelihood   BIC:                         1.322e+04
Date:                Thu, 26 Oct 2023                                         
Time:                        17:34:35                                         
No. Observations:                1372                                         
Df Residuals:                    1264                                         
Df Model:                          11                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
TitleFlag           0.6330      0.223      2.842      0.004       0.196       1.070
TitleDensity        0.5903      0.204      2.895      0.004       0.191       0.990
URLFlag            -0.0890      0.234     -0.381      0.703      -0.547       0.369
URLDensity          1.3060      0.269      4.854      0.000       0.779       1.833
MetaFlag           -0.1294      0.075     -1.730      0.084      -0.276       0.017
MetaDensity         0.8170      0.423      1.929      0.054      -0.013       1.647
PageAuthority       0.0016      0.012      0.137      0.891      -0.022       0.025
DomainAuthority     0.0037      0.005      0.791      0.429      -0.005       0.013
LinkingDomain      -0.0133      0.061     -0.218      0.827      -0.133       0.106
InboundLink        -0.0191      0.031     -0.622      0.534      -0.079       0.041
RankingKeyword      0.0414      0.020      2.028      0.043       0.001       0.081
3/4                -1.7461      0.161    -10.845      0.000      -2.062      -1.431
4/5                -1.2722      0.261     -4.872      0.000      -1.784      -0.760
5/6                -1.7326      0.262     -6.622      0.000      -2.245      -1.220
6/7                -2.0233      0.262     -7.709      0.000      -2.538      -1.509
7/8                -2.2335      0.263     -8.490      0.000      -2.749      -1.718
8/9                -2.3962      0.264     -9.093      0.000      -2.913      -1.880
9/10               -2.5229      0.264     -9.563      0.000      -3.040      -2.006
10/11              -2.6232      0.264     -9.935      0.000      -3.141      -2.106
11/12              -2.7155      0.264    -10.275      0.000      -3.233      -2.198
12/13              -2.8004      0.264    -10.589      0.000      -3.319      -2.282
13/14              -2.8741      0.265    -10.860      0.000      -3.393      -2.355
14/15              -2.9390      0.265    -11.100      0.000      -3.458      -2.420
15/16              -2.9934      0.265    -11.301      0.000      -3.513      -2.474
16/17              -3.0443      0.265    -11.489      0.000      -3.564      -2.525
17/18              -3.0910      0.265    -11.662      0.000      -3.610      -2.571
18/19              -3.1337      0.265    -11.819      0.000      -3.653      -2.614
19/20              -3.1773      0.265    -11.980      0.000      -3.697      -2.657
20/21              -3.2160      0.265    -12.123      0.000      -3.736      -2.696
21/22              -3.2487      0.265    -12.244      0.000      -3.769      -2.729
22/23              -3.2792      0.265    -12.357      0.000      -3.799      -2.759
23/24              -3.3089      0.265    -12.466      0.000      -3.829      -2.789
24/25              -3.3370      0.265    -12.570      0.000      -3.857      -2.817
25/26              -3.3610      0.266    -12.659      0.000      -3.881      -2.841
26/27              -3.3824      0.266    -12.738      

In [63]:
X_test.shape[1]

11

In [64]:
# Make prediction: assume the predicted output of this model is y_pred,
# for this question, you need to do y_pred.argmax(1)+3 to get the real output.
# Please refer to the reference above for how to make prediction properly.
y_pred_probabilities = results.predict(X_test)
predictions = y_pred_probabilities.to_numpy().argmax(1) + 3

In [65]:
# Print RMSE, y_test is a numpy array from test_label, y_pred is a numpy array from your model prediction
resids = y_test - predictions
mse = np.mean(resids**2)

rmse = np.sqrt(mse)

print("RMSE:", rmse)

RMSE: 53.18070209041934


Pick the most statistically significant variable and interpret its estimated beta coefficient. (5 points)

*ANSWER: HERE*
Both URLDensity and DomainAuthority are statistically significant, both with P-values close to 0. DomainAuthority has such low P-values in this case but its 95%CI is closer 0. For a 1 unit increase in DomainAuthority, it gives a positive change in dependent variable ReverseRank by 0.0133, holding all other variables constant.

URLDensity 0.775	1.825

DomainAuthority	0.007	0.019

### Q3. Decision Tree (5 points)
Please use the training dataset to fit a decision tree model with all variables aforementioned. **Use the `ReverseRank` as the target variable**.


In [66]:
# Data Input
X_train = train_feature # trainig feature, as a dataframe in pandas
X_test = test_feature # test feature, as a df

y_train = train_label.values # training label, as a numpy array
y_test = test_label.values # test label, as a np array

In [70]:
# Model & Fit: USE DecisionTreeClassifier(random_state=0).fit(...)
# y_train is a numpy array from train_label, X_train is a pandas dataframe from train_feature.
# Make prediction
results = DecisionTreeRegressor(random_state=0)

# Fit the classifier to the training data
results.fit(X_train, y_train)

# Make predictions on the test data
predictions = results.predict(X_test)

In [71]:
# Print RMSE, y_test is a numpy array from test_label, y_pred is a numpy array from your model prediction
resids = y_test - predictions
mse = np.mean(resids**2)

rmse = np.sqrt(mse)

print("RMSE:", rmse)

RMSE: 42.57505876979738


###  Q4. Random Forest (5 points)
Please use the training dataset to fit a random forest model with all variables aforementioned. **Use the `ReverseRank` as the target variable**.


In [75]:
# Data Input
X_train = train_feature # trainig feature, as a dataframe in pandas
X_test = test_feature # test feature, as a df

y_train = train_label.values # training label, as a numpy array
y_test = test_label.values # test label, as a np array

In [76]:
# Model & Fit: USE RandomForestRegressor(random_state=0).fit(...)
# y_train is a numpy array from train_label, X_train is a pandas dataframe from train_feature.
# Make prediction
results = RandomForestRegressor(random_state=0)

# Fit the classifier to the training data
results.fit(X_train, y_train)

# Make predictions on the test data
predictions = results.predict(X_test)

<ipython-input-76-791b7e75c08f>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  results.fit(X_train, y_train)


In [77]:
# Print RMSE, y_test is a numpy array from test_label, y_pred is a numpy array from your model prediction
resids = y_test - predictions
mse = np.mean(resids**2)

rmse = np.sqrt(mse)

print("RMSE:", rmse)

RMSE: 31.89853003853016


## Problem 2. Pairwise Rank with `xgboost` (35 points)
Please use the training dataset to fit an XGboost model with all variables aforementioned. **Use the `ReverseRank` as the target variable**.

Show the feature importance plot. Use "Gain" for `measure` in the importance plot.

For the XGboost prediction, please interpret the importance value of the most important variable. Please use the trained XGboost model to predict the rank in the test dataset and report the RMSE of the prediction.

Reference: https://xgboost.readthedocs.io/en/latest/python/index.html

In [ ]:
# Data Input


### Q1. Use `reg:linear` for `objective` and `rmse` for `eval_metric`. (10 points)


In [ ]:
# Model and Parameter Setting
# Use xgb.XGBRegressor
# Use parameters below first and you can adjust it later.
reg = xgb.XGBRegressor(max_depth = 6,
                       eta = 0.1,
                       gamma = 0.1,
                       subsample = 0.8,
                       colsample_bytree = 0.8,
                       alpha = 0.5,
                       objective = "reg:linear",
                       eval_metric = "rmse",
                       seed = 1)

In [ ]:
# Fit the model by your_model.fit(X_train, y_train)


In [ ]:
# Show the feature importance plot. Use plot_importance(your_model, importance_type = 'gain')
# If you don't see your plot, please run this cell again.


**Please interpret the importance value of the most important variable.** (5 points)

*ANSWER: HERE*


In [ ]:
# Make prediction: y_pred should be the output of your prediction, it cotains value
# from 3 to 100 (may not be integers)


In [ ]:
## Print RMSE, y_test is a numpy array from test_label, y_pred is a numpy array from your model prediction



**Try different combinations of the hyper-parameters to get a lower RMSE.** (5 points)

Please note that you can use any hyper-parameter tuning techniques in the lecture. The points are not awarded based on the exact number of RMSE. As long as it is lower than the original values and you can justify the technique you use, you will be awarded the points.

In [ ]:
# your code here...


### Q2. Redo Q1 by using `rank:pairwise` for `objective` and `rmse` for `eval_metric`. (15 points)
Observe that we did not utilize the query information yet. However, in the dataset, we know that each query corresponds to 98 data points. Now we will leverage this information by setting group information for both training and testing data. (5 points)



In [ ]:
# Model and Parameter Setting
# Use xgb.XGBRanker
# Use parameters below first and you can adjust it later.
reg = xgb.XGBRanker(max_depth = 6,
                           eta = 0.1,
                           gamma = 0.1,
                           subsample = 0.8,
                           colsample_bytree = 0.8,
                           alpha = 0.5,
                           objective = "rank:pairwise",
                           eval_metric = "rmse")

In [ ]:
# Data Input


In [ ]:
# Fit the model by your_model.fit(X_train, y_train, group = np.full(14,98))


In [ ]:
# Show the feature importance plot. Use plot_importance(your_model, importance_type = 'gain')


**Please interpret the importance value of the most important variable.** (5 points)

*ANSWER: HERE*


Note that here `y_pred` is *not* a direct rank on itself. We need to figure out how to get the rank from `y_pred`. Test data can be divided into 6 groups, we need to perform the following steps for each group.
1. For each group, get `y_pred` from your model.
2. The values in `y_pred` indicate a relative score for the rank. The higher the score, the better the page rank. For all 98 pages in one group, the page with the highest score should rank as 100, and the page with the lowest score should rank as 3.
3. Repeat this for all groups. You are required to implement this by some simple codes. **y_pred_rank** is the predicted ranks you transformed from **y_pred**.



In [ ]:
# Make prediction: y_pred is the predicted output from your model, y_pred_rank is the actual rank you get


In [ ]:
# Print RMSE, y_test is a numpy array from test_label, y_pred_rank is a numpy array from your model prediction


**Try different combinations of the hyper-parameters to get a lower RMSE.** (5 points)

Please note that you can use any hyper-parameter tuning techniques in the lecture. The points are not awarded based on the exact number of RMSE. As long as it is lower than the original values and you can justify the technique you use, you will be awarded the points.

In [ ]:
# your code here...


### Q3. Use `rank:ndcg` for `objective` and `rmse` for `eval_metric`. Redo Question 6. (10 points)

In [ ]:
# Model and Parameter Setting
# Use xgb.XGBRanker
# Use parameters below first and you can adjust it later.
reg = xgb.XGBRanker(max_depth = 6,
                           eta = 0.1,
                           gamma = 0.1,
                           subsample = 0.8,
                           colsample_bytree = 0.8,
                           alpha = 0.5,
                           objective = "rank:ndcg",
                           eval_metric = "rmse")

In [ ]:
# Data Input


In [ ]:
# Fit the model by your_model.fit(X_train, y_train, group = np.full(14,98))


In [ ]:
# Show the feature importance plot. Use plot_importance(your_model, importance_type = 'gain')


**Please interpret the importance value of the most important variable.** (5 points)

*ANSWER: HERE*


In [ ]:
# Make prediction: y_pred is the predicted output from your model, y_pred_rank is the actual rank you get


In [ ]:
# Print RMSE, y_test is a numpy array from test_label, y_pred_rank is a numpy array from your model prediction


**Try different combinations of the hyper-parameters to get a lower RMSE.** (5 points)

Please note that you can use any hyper-parameter tuning techniques in the lecture. The points are not awarded based on the exact number of RMSE. As long as it is lower than the original values and you can justify the technique you use, you will be awarded the points.

In [ ]:
# your code here...


## Problem 3. Interpretation Short-Answers (10 points)

Please paste the RMSE values of all questions above in the table below. Please round your RMSE up to 4 decimals.

To access the table written in markdown below, double click the placeholder table, edit the corresponding value and run the cell.


Before Tuning:

| Question | P1-Q1 | P1-Q2 | P1-Q3 | P1-Q4 | P2-Q5 | P2-Q6 | P2-Q7 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| RMSE | 1.0000 | 2.0000 | 3.0000 | 4.0000 | 5.0000 | 6.0000 | 7.0000 |

After Tuning:

| Question | P2-Q5 | P2-Q6 | P2-Q7 |
| --- | --- | --- | --- |
| RMSE |  5.0000 | 6.0000 | 7.0000 |

**Question: Please describe what you observe from the RMSE value for all methods of pointwise ranking. What is the best method? Why? (5 points)**

Answer: HERE

**Question: Please compare the results from pointwise ranking and pairwise ranking and describe your observatins.(5 points)**

Answer: HERE

# Appendix

1. ID: identification number (i.e., row number in the dataset)

2. Position: the actual google ranking of the webpage to the query

3. ReverseRank: equals $101$ minus Position. It is equivalent to the position. Sometimes using ReverseRank as the dependent variable can have a better prediction.

4. Title: the title of the webpage

5. URL: the URL of the webpage

6. Meta: the meta description of the webpage

7.	TitleFlag: indicates that whether the whole keyword is included in the page title. TitleFlag equals 1 if yes, otherwise, 0.

8.	UrlFlag: indicates that whether the whole keyword is included in the page url. UrlFlag equals 1 if yes, otherwise, 0.

9.	MetaFlag: indicates that whether the whole keyword is included in the page meta description. MetaFlag equals 1 if yes, otherwise, 0.

10.	TitleDensity: is the percentage of times a keyword appears in the title of a web page compared to the total number of words in the title of a web page.

11.	UrlDensity: is the percentage of times a keyword appears in the URL of a web page compared to the total number of words in the URL of a web page.

12.	MetaDensity: is the percentage of times a keyword appears in the meta description of a web page compared to the total number of words in the meta description of a web page.

13.	PageAuthority: is a score developed by Moz that predicts how well a specific page will rank on search engine result pages (SERP). https://moz.com/learn/seo/page-authority

14.	DomainAuthority: is a search engine ranking score developed by Moz that predicts how well a website will rank on search engine result pages (SERPs). https://moz.com/learn/seo/domain-authority

15.	LinkingDomain: is the number of unique external domains linking to this page. Two or more links from the same websites are considered as one linking domain. Provided by Moz.

16.	InboundLink: is the number of unique external pages linking to this page. Two or more links from the same page on a website are considered as one inbound link. Provided by Moz.

17.	RankingKeyword: is the number of keywords for which this site ranks within the top 50 positions on Google US. Provided by Moz.